In [12]:
import pandas as pd

# Carregar os arquivos CSV
precos = pd.read_csv("../data/precos_acoes.csv")
div = pd.read_csv("../data/dividendos_ano_resumo.csv")

# Normalizar tickers para fazer o merge (remover .SA e espaços, tudo maiúsculo)
precos["ticker_base"] = (
    precos["ticker"].astype(str).str.upper().str.strip().str.replace(".SA", "", regex=False)
)
div["ticker_base"] = div["ticker"].astype(str).str.upper().str.strip()

# Garantir que os numéricos estão como número
for col in ["fechamento_atual"]:
    if col in precos.columns:
        precos[col] = pd.to_numeric(precos[col], errors="coerce")

for col in ["valor_5anos", "valor_12m"]:
    if col in div.columns:
        div[col] = pd.to_numeric(div[col], errors="coerce")

# Unir pelo ticker_base
df = div.merge(precos[["ticker_base", "fechamento_atual"]], on="ticker_base", how="left")

# Calcular DYs (evitar divisão por zero/NaN)
df["DY5anos"] = (((df["valor_5anos"] / 5) / df["fechamento_atual"]) * 100).where(df["fechamento_atual"] > 0)
df["DY12M"]   = ((df["valor_12m"] / df["fechamento_atual"]) * 100).where(df["fechamento_atual"] > 0)

# (Opcional) arredondar e ordenar
df["DY5anos"] = df["DY5anos"].round(2)
df["DY12M"]   = df["DY12M"].round(2)
df_ordenado = df.sort_values("DY12M", ascending=False)

# Ver um pedaço
print(df_ordenado.head(20)[["ticker", "fechamento_atual", "valor_12m", "valor_5anos", "DY12M", "DY5anos"]])

# Salvar se quiser
# df_ordenado.to_csv("dy_resultado.csv", index=False)


     ticker  fechamento_atual  valor_12m  valor_5anos   DY12M  DY5anos
118   TIMS3             22.37  84.849915   342.154002  379.30   305.90
85    MELK3              3.37   0.978435     2.431989   29.03    14.43
87    MOAR3            130.00  32.241685   148.679052   24.80    22.87
52    DIRR3             13.66   1.936667     5.396154   14.18     7.90
112   SOND6             50.01   6.271946    32.561482   12.54    13.02
69    GEPA4             35.20   4.048288    17.357916   11.50     9.86
111   SOND5             54.80   6.271946    32.561482   11.45    11.88
63    EQPA3              5.76   0.658700     2.837035   11.44     9.85
68    GEPA3             35.98   4.048288    17.357916   11.25     9.65
9     BAZA3             73.99   8.012259    29.582096   10.83     8.00
16    BGIP4             27.95   2.985803    11.338142   10.68     8.11
97   RBIF11             72.11   7.430000    33.070000   10.30     9.17
40    CMIG4             10.67   1.056537     5.750873    9.90    10.78
74    

In [15]:
import pandas as pd
import os

# Carregar os arquivos CSV
precos = pd.read_csv("../data/precos_acoes.csv")
div = pd.read_csv("../data/dividendos_ano_resumo.csv")

# Normalizar tickers para merge
precos["ticker_base"] = (
    precos["ticker"].astype(str).str.upper().str.strip().str.replace(".SA", "", regex=False)
)
div["ticker_base"] = div["ticker"].astype(str).str.upper().str.strip()

# Garantir tipos numéricos
precos["fechamento_atual"] = pd.to_numeric(precos["fechamento_atual"], errors="coerce")
div["valor_5anos"] = pd.to_numeric(div["valor_5anos"], errors="coerce")
div["valor_12m"] = pd.to_numeric(div["valor_12m"], errors="coerce")

# Merge para ter todos os tickers do arquivo de preços
df = precos.merge(div[["ticker_base", "valor_5anos", "valor_12m"]], on="ticker_base", how="left")

# Calcular DYs
df["DY5anos"] = (((df["valor_5anos"] / 5) / df["fechamento_atual"]) * 100).where(df["fechamento_atual"] > 0)
df["DY12M"]   = ((df["valor_12m"] / df["fechamento_atual"]) * 100).where(df["fechamento_atual"] > 0)

# Arredondar
df["DY5anos"] = df["DY5anos"].round(2)
df["DY12M"] = df["DY12M"].round(2)

# Selecionar apenas as colunas desejadas
df_final = df[["ticker", "DY5anos", "DY12M"]]

# Mostrar primeiros 5
print(df_final.head(5))

# Salvar no caminho solicitado
os.makedirs("../data", exist_ok=True)
df_final.to_csv("../data/dividend_yield.csv", index=False)
print("Arquivo salvo em: ../data/dividend_yield.csv")


     ticker  DY5anos   DY12M
0  VIVT3.SA     3.71    3.11
1  TIMS3.SA   305.90  379.30
2  FIQE3.SA     3.23    2.53
3  BRST3.SA     1.45    7.27
4  TELB4.SA      NaN     NaN
Arquivo salvo em: ../data/dividend_yield.csv
